In [ ]:
%%writefile Sequential.cu
#include <stdio.h>
#include <malloc.h>
#include <math.h>
#define Nx 16
#define Ny 16
#define Nt 21
#define dxy 0.1
#define dt 0.01
//==============================
float funct1(int i, int j) {
  return sin(M_PI*(i*dxy + j*dxy));
}
float funct2(int i, int j) {
  return cos(M_PI*(i*dxy + j*dxy));
}
//==============================
void InputData(float *U, float *V, float *C) {
  int i,j,k;
  for (j=0; j<Ny; j++) {
    for (i=0; i<Nx; i++) {
      *(U + j*Nx + i) = funct1(i,j);
      *(V + j*Nx + i) = dt*funct2(i,j) + *(U + j*Nx + i);
    }
  }
  for (k=0; k<Nt; k++) {
    *(C+k) = 1/2;
  }
}
//===============================
void Computing(float *U, float *V, float *C) {
  int i,j,k, nt_half;
  float left, right, bottom, top, center1, center2;
  nt_half = (Nt+1)/2;
  for (k=1; k<nt_half; k++){
    for (j=0; j<Ny; j++) {
      for (i=0; i<Nx; i++) {
        left  =  (i == 0)    ? 0 : *(V + j * Nx + i-1);
        right =  (i == Nx-1) ? 0 : *(V + j * Nx + i+1);
        bottom = (j == 0)    ? 0 : *(V + (j+1) * Nx + i);
        top   =  (j == Nx-1) ? 0 : *(V + (j-1) * Nx + i);
        center1 = *(V + j * Nx + i);
        center2 = *(U + j * Nx + i);
        *(U + j * Nx + i) = 2*center1 - center2 + ((dt * dt * (*(C + k)) * (*(C + k))) / (dxy * dxy)) * (left + right + bottom + top - 4*center1);
      }
    }
    for (j=0; j<Ny; j++) {
      for (i=0; i<Nx; i++) {
        left  =  (i == 0)    ? 0 : *(U + j * Nx + i-1);
        right =  (i == Nx-1) ? 0 : *(U + j * Nx + i+1);
        bottom = (j == 0)    ? 0 : *(U + (j-1) * Nx + i);
        top   =  (j == Nx-1) ? 0 : *(U + (j+1) * Nx + i);
        center1 = *(U + j * Nx + i);
        center2 = *(V + j * Nx + i);
        *(V + j * Nx + i) = 2*center1 - center2 + ((dt * dt * (*(C + k)) * (*(C + k))) / (dxy * dxy)) * (left + right + bottom + top - 4*center1);
      }
    }
  }
}
//===============================
int main() {
  float *U, *V, *C;
  U = (float *) malloc (Nx*Ny*sizeof(float));
  V = (float *) malloc (Nx*Ny*sizeof(float));
  C = (float *) malloc (Nt*sizeof(float));
  InputData(U, V, C);
  Computing(U, V, C);
  int i, j;
  if (Nt%2 == 1) {
    for (i=0; i<Nx; i++) {
      for (j=0; j<Ny; j++) {
        printf("%f ", *(U + j*Nx + i));
      }
      printf("\n");
    }
  }
  else {
    for (i=0; i<Nx; i++) {
      for (j=0; j<Ny; j++) {
        printf("%f ", *(V + j*Nx + i));
      }
      printf("\n");
    }
  }
}

In [59]:
!nvcc Sequential.cu -lm

In [ ]:
!./a.out